In [1]:
from multiprocessing import Process, Queue
import glob
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import math as m
import numpy as np
%matplotlib inline
plt.style.use('ggplot')

def collect_sqlm(num1,num2,queue):
    for i in range (num1,num2):
        tablenames = []
        operand_size = 0
        index_size = 0
        log_os = 0
        log_is = 0
        regex =[]
        query = fdf.loc[i,('query')].upper()
        query = re.sub(r',\s+', ',', query)
        if re.search(r'\ASELECT|INSERT|DELETE',query):
            regex = re.findall(r'(?<=FROM|JOIN|INTO)\s+[\w.]+',query)
        if re.search(r'\AUPDATE',query):
            regex = re.findall(r'(?<=UPDATE)\s+[\w.]+',query)
        if re.search(r'\AMERGE',query):
            regex1 = re.findall(r'(?<=INTO)\s+[\w.]+',query)
            regex2 = re.findall(r'(?<=MERGE)\s+[\w.]+',query)
            regex = regex1+regex2
        for item in regex:
            for table in item.split(','):
                tablenames.append(table)
        for table in tablenames:
            tableschema = ''
            tablename = ''
            if len(table.split('.')) == 2:
                tableschema = table.split('.')[0]
                tablename = table.split('.')[1]
                try:
                    operand_size = operand_size + sizedf.loc[tableschema.strip(),tablename.strip()].object_size
                    
                except KeyError:
                    operand_size = 0
                try:
                    index_size = index_size + sizedf.loc[tableschema.strip(),tablename.strip()].index_size
                except KeyError:
                    index_size = 0
        num_operations = 0
        
        base_operation = 'SELECT|INSERT|DELETE|UPDATE|MERGE|VALUES'
        set_operation = 'JOIN|UNION'
        final_operation = 'ORDER BY| GROUP BY'
        regex = re.findall(base_operation,query)
        num_operations = num_operations+len(regex)
        regex = re.findall(set_operation,query)
        num_operations = num_operations+len(regex)
        regex = re.findall(final_operation,query)
        num_operations = num_operations+len(regex)
        
        num_filters = 0
        regex = re.findall(r'WHERE',query)
        num_filters = len(regex)
        regex = re.findall(r' AND | OR ',query)
        num_filters = num_filters + len(regex)
        
        objects_size_tuple = (i,operand_size/1024/1024,index_size/1024/1024,num_operations,num_filters)
        queue.put(objects_size_tuple)
    queue.put((-1,-1,-1,-1,-1))
    return

def read_queue(queue):
    global sizobj_list
    global num_procs
    count = 0
    while True:
            location , operand_size,index_size,num_operations,num_filters = queue.get()
            if (operand_size == -1):
                count = count + 1
                #print (count)
            else:
                sizeobj_list.append((location,operand_size,index_size,num_operations,num_filters))
            if (count == num_procs):
                print ("done")
                break
    return

In [2]:
path = r'/data/developer/python/test'    
allFiles = glob.glob(path + "/SQL_Metrics*.csv")     
list_ = []
notvalues = []
for file_ in allFiles:
    regex = re.search('SQL_Metrics_([0-9]*-[0-9]*-[0-9]*)-([0-9]*)', file_)
    df = pd.read_csv(file_,sep = '@',names = ['stmt_hash','query','exec_time'],index_col=False)
    df['date'] = regex.group(1)
    df['hour'] = regex.group(2)
    df.exec_time = pd.to_numeric(df.exec_time,errors='coerce')
    df.hour = pd.to_numeric(df.hour,errors='coerce')
    list_.append(df)
fdf = pd.concat(list_,ignore_index=True)
fdf = fdf[fdf.exec_time > 0].reset_index().drop('index',axis=1)       



allFiles = glob.glob(path + "/size*.csv")     
list_ = []
for file_ in allFiles:
    regex = re.search('size*', file_)
    sizedf = pd.read_csv(file_,sep = '@',names = ['tableschema','tablename','object_size','index_size','lob_size'],index_col=False)
    sizedf = sizedf.set_index(['tableschema', 'tablename'])

plt.style.use('ggplot')



fields = ['time','lock_wait_time','total_section_sort_time','diaglog_write_wait_time','direct_read_time','direct_write_time','log_buffer_wait_time','log_disk_wait_time','pool_read_time','pool_write_time','prefetch_wait_time','total_act_time','total_act_wait_time','total_cpu_time','total_extended_latch_wait_time','span']
path = r'/data/developer/python/test'    
allFiles = glob.glob(path + "/workload_stats_*.csv")    
list_ = []
for file_ in allFiles:
    regex = re.search('workload_stats_([0-9]*-[0-9]*-[0-9]*)', file_)
    wldf = pd.read_csv(file_,sep = '@',names = fields,index_col=False)
    list_.append(wldf)
final_wldf = pd.concat(list_,ignore_index=True)
final_wldf['hour'] = 0
final_wldf['date'] = 0
for i in range (0,len(final_wldf)):
    regex = re.search('([0-9-]*)T([0-9]*)', final_wldf.loc[i,('time')])
    final_wldf.loc[i,('date')] = regex.group(1)
    final_wldf.loc[i,('hour')] = int(regex.group(2)) + 1 
    
final_wldf = final_wldf.set_index(['date', 'hour'])
final_wldf.drop('time',axis=1,inplace = True)
final_wldf.drop('span',axis=1,inplace = True)

In [4]:
final_wldf['log_total_cpu_time'] = 0
for i in range (0,len(final_wldf)):
    if (final_wldf.iloc[i].total_cpu_time!=0):
           final_wldf.iloc[i,final_wldf.columns.get_loc('log_total_cpu_time')] = m.log2(final_wldf.iloc[i].total_cpu_time)


In [5]:
import time
queue = Queue()
num_procs = 8
sizeobj_list = []
pro_list = []
num_rows = len(fdf)
group_size = int(num_rows/num_procs)
for group_num in range(0,num_procs):
    num1 = group_num*(group_size)
    num2 = num1 + group_size - 1
    proc = Process(target=collect_sqlm, args=(num1,num2,queue))
    pro_list.append(proc)

start = time.time()
for p in pro_list:
    p.start()
    
read_queue(queue)

sizeobj_df = pd.DataFrame(sizeobj_list, columns=['location', 'operand_size','index_size','num_operations','num_filters']).set_index('location')

final_df_sqlm = fdf.merge(sizeobj_df, left_index = True, right_index=True, how='inner').reset_index().drop('index',axis = 1)

sqlm_df = final_df_sqlm[final_df_sqlm.operand_size != 0]

sqlm_df = sqlm_df.set_index(['date', 'hour'])

#sqlm_final_df = sqlm_df.groupby(['date','hour'])['operand_size','index_size','num_operations','num_filters','exec_time'].mean()

final_df_test = sqlm_df.merge(final_wldf, left_index = True, right_index=True, how='inner')
print(final_df_test.shape)

processtime = time.time() - start
print("Took {} seconds with {} process(es)".format(processtime,num_procs))

done
(122430, 22)
Took 18.440097093582153 seconds with 8 process(es)


In [6]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

%run train.ipynb


#ypred = olsres.predict(X)
#print(ypred)

done
Took 159.5001299381256 seconds with 8 processes.


In [7]:
print(est.mse_resid)
print(est.mse_total)

0.0265429359363
8.91158293486


In [8]:
final_df_test['predicted_exec_time'] = 0
X = final_df_test [['operand_size','num_operations','log_total_cpu_time']]
Y = est.predict(X)
for i in range (0,len(final_df_test)):
 final_df_test.iloc[i,final_df_test.columns.get_loc('predicted_exec_time')] = 2**Y[i]

In [9]:
predicted_df = final_df_test[['query','exec_time','predicted_exec_time']]
predicted_df = predicted_df.reset_index()

In [10]:
predicted_df['error'] = 0
for i in range (0,len(predicted_df)):
    residual = predicted_df.iloc[i].exec_time - predicted_df.iloc[i].predicted_exec_time
    predicted_df.iloc[i,predicted_df.columns.get_loc('error')] = residual

In [11]:
predicted_df = predicted_df[predicted_df.error<1]

In [12]:
predicted_df[['query','exec_time','predicted_exec_time','error']]

,query,exec_time,predicted_exec_time,error
0,UPDATE Fulfillment.ServerIndexStatus_2 SET D...,0.040323,0.338453,-0.298131
1,"SELECT GroupId, ServerName, DatabaseName, Gene...",0.125000,0.338006,-0.213006
2,SELECT Count(1) FROM Contract.IncrIndexingQueue_9,0.050000,0.338006,-0.288006
3,"SELECT FolderId, DocumentId, State, WaitingOnF...",0.057292,0.404207,-0.346915
4,INSERT INTO FULFILLMENT.ORDERWFACTIVITYHISTORY...,0.375000,0.046738,0.328262
5,insert into Requisition.CFValueSetDigest (CFVA...,0.571429,0.001215,0.570213
6,SELECT min(CreateDate) FROM ContractAttachment...,0.050000,0.338006,-0.288006
7,"SELECT ResponseId,EventId,SupplierId,PrebidRes...",0.027778,0.045915,-0.018137
8,"SELECT SI. ItemId , SI.OrgId,SI.ManufacturerI...",0.039088,0.351823,-0.312735
9,"SELECT ScheduleId, OrgId, InternalName, Schedu...",0.076923,0.338006,-0.261083


In [80]:
print (sum(predicted_df.error))
print (len(predicted_df))

9651.62097455
100396
